In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# %pwd

In [3]:
# %cd /content/drive/My\ Drive/DQN
FOLDER_PATH = '/home/taindp/Jupyter/custom_dqn'
# FOLDER_PATH = '/home/taindp/Jupyter/deep_q_learning'
# FOLDER_PATH = %pwd
# FOLDER_PATH = '/home/waves8/taindp/deep_q_learning'
CONSTANTS_FILE_PATH = f'{FOLDER_PATH}/constants.json'
# SAVE_WEIGHT_PATH =f'weight/goal400_10.h5'
%cd $FOLDER_PATH
!ls

/home/taindp/Jupyter/custom_dqn
check_ref.ipynb		   LICENSE		train.py
constants.json		   log3.txt		userold.py
create_goal.ipynb	   pickle_converter.py	user.py
data			   __pycache__		user_simulator_modify_taindp.py
db_query.py		   requirements.txt	user_simulator.py
dialogue_config.py	   state_tracker.py	utils.py
dqn_agent.py		   test.py		weight
error_model_controller.py  train_dqn.ipynb


In [4]:
%load_ext autoreload
%autoreload 2
from user_simulator import UserSimulator
from error_model_controller import ErrorModelController
from dqn_agent import DQNAgent
# from dqn_agent import get_action
from state_tracker import StateTracker
import pickle, argparse, json, math
from utils import remove_empty_slots
from user import User
import time
import json
from tqdm import tqdm
from db_query import DBQuery
from datetime import date
today = str(date.today().strftime("%d/%m/%Y")).replace(r'/',r'_')

Using TensorFlow backend.


In [5]:
def run_round(state, warmup=False):
    # 1) Agent takes action given state tracker's representation of dialogue (state)
    agent_action_index, agent_action = dqn_agent.get_action(state, use_rule=warmup)
#     print('agent_action_index',agent_action_index)
#     print('agent_action',agent_action)
    # 2) Update state tracker with the agent's action
    state_tracker.update_state_agent(agent_action)
#     print('current round',agent_action['round'])
    # 3) User takes action given agent action
    user_action, reward, done, success = user.step(agent_action)
#     print('user_action',user_action)
#     print('reward',reward)
#     print('done',done)
#     print('success',success)
    if not done:
        # 4) Infuse error into semantic frame level of user action
        emc.infuse_error(user_action)
#         print('user_action emc',user_action)
    # 5) Update state tracker with user action
    state_tracker.update_state_user(user_action)
#     print('update_state_user',user_action)
    # 6) Get next state and add experience
    next_state = state_tracker.get_state(done)
#     print('next_state',next_state)
    dqn_agent.add_experience(state, agent_action_index, reward, next_state, done)
    return next_state, reward, done, success


def warmup_run():
    """
    Runs the warmup stage of training which is used to fill the agents memory.

    The agent uses it's rule-based policy to make actions. The agent's memory is filled as this runs.
    Loop terminates when the size of the memory is equal to WARMUP_MEM or when the memory buffer is full.

    """

    print('Warmup Started...')
    total_step = 0
    with tqdm(total=1) as pbar:
        while total_step != WARMUP_MEM and not dqn_agent.is_memory_full():
            # Reset episode
            episode_reset()
            done = False
            # Get initial state from state tracker
            state = state_tracker.get_state()
            # print('State warmup', state)
            # print('Type state',type(state))
            while not done:
                next_state, _, done, _ = run_round(state, warmup=True)
#                 next_state, _, done, _ = run_round(state)
                total_step += 1
                state = next_state
            # print(total_step)
            # time.sleep(1)
            pbar.update(1)
    print('...Warmup Ended')


def train_run():
    """
    Runs the loop that trains the agent.

    Trains the agent on the goal-oriented chatbot task. Training of the agent's neural network occurs every episode that
    TRAIN_FREQ is a multiple of. Terminates when the episode reaches NUM_EP_TRAIN.

    """

    print('Training Started...')
    episode = 0
    period_reward_total = 0
    period_success_total = 0
    success_rate_best = 0.8
    # with tqdm(total=NUM_EP_TRAIN) as pbar1:
    while episode < NUM_EP_TRAIN:
        episode_reset()
        episode += 1
#         print('Current episode:',episode)
        done = False
        state = state_tracker.get_state()
        # print('Current state',state)
        while not done:
            next_state, reward, done, success = run_round(state)
            # print('Next state',next_state)
#             print('Reward', reward)
#             print('Done',done)
            period_reward_total += reward
            state = next_state
#         print("success :{0}".format(success))

        period_success_total += success
        # print('period_success_total',period_success_total)
        # Train
        if episode % TRAIN_FREQ == 0:
            print('Current episode:',episode)
            # Check success rate
            success_rate = period_success_total / TRAIN_FREQ
            avg_reward = period_reward_total / TRAIN_FREQ
            print("episode {0}: success rate: {1}".format(episode, success_rate))

            # Flush
            if success_rate >= success_rate_best and success_rate >= SUCCESS_RATE_THRESHOLD:
                dqn_agent.empty_memory()
            # Update current best success rate
            if success_rate >= success_rate_best:
                print('Episode: {} NEW BEST SUCCESS RATE: {} Avg Reward: {}' .format(episode, success_rate, avg_reward))
                success_rate_best = success_rate
                dqn_agent.save_weights()
            period_success_total = 0
            period_reward_total = 0
            # Copy
            dqn_agent.copy()
            # Train
            dqn_agent.train()
          # pbar1.update(1)
    print('...Training Ended')


def episode_reset():
    """
    Resets the episode/conversation in the warmup and training loops.

    Called in warmup and train to reset the state tracker, user and agent. Also get's the initial user action.

    """

    # First reset the state tracker
    state_tracker.reset()
    # Then pick an init user action
    user_action = user.reset()
    # Infuse with error
    emc.infuse_error(user_action)
    # And update state tracker
    state_tracker.update_state_user(user_action)
    # Finally, reset agent
    dqn_agent.reset()

In [6]:
constants_file = CONSTANTS_FILE_PATH

with open(constants_file) as f:
    constants = json.load(f)

In [7]:
constants

{'db_file_paths': {'database': 'data/db_hcmut_600.json',
  'dict': 'data/db_entity_jan23.json',
  'user_goals': 'data/goal_hcmut_600.json'},
 'run': {'usersim': True,
  'warmup_mem': 2600,
  'num_ep_run': 25000,
  'train_freq': 100,
  'max_round_num': 15,
  'success_rate_threshold': 0.2},
 'agent': {'save_weights_file_path': '',
  'load_weights_file_path': 'weight/23_01_2021_25000_100_15.h5',
  'vanilla': True,
  'learning_rate': 0.0005,
  'batch_size': 64,
  'dqn_hidden_size': 100,
  'epsilon_init': 0.0,
  'gamma': 0.9,
  'max_mem_size': 500000},
 'emc': {'slot_error_mode': 0,
  'slot_error_prob': 0.1,
  'intent_error_prob': 0.0}}

In [8]:
# Load file path constants
file_path_dict = constants['db_file_paths']
DATABASE_FILE_PATH = file_path_dict['database']
DICT_FILE_PATH = file_path_dict['dict']
USER_GOALS_FILE_PATH = file_path_dict['user_goals']

In [9]:
run_dict = constants['run']
USE_USERSIM = run_dict['usersim']
WARMUP_MEM = run_dict['warmup_mem']
NUM_EP_TRAIN = run_dict['num_ep_run']
TRAIN_FREQ = run_dict['train_freq']
MAX_ROUND_NUM = run_dict['max_round_num']
SUCCESS_RATE_THRESHOLD = run_dict['success_rate_threshold']

In [10]:
WARMUP_MEM

2600

In [11]:
NUM_EP_TRAIN

25000

In [12]:
constants['agent']['save_weights_file_path'] = 'weight/' + str(today)+ '_' +str(NUM_EP_TRAIN)+ '_'+str(TRAIN_FREQ)+'_'+str(MAX_ROUND_NUM) + '.h5'

In [13]:
# constants

In [14]:
database= json.load(open(DATABASE_FILE_PATH,encoding='utf-8'))

In [15]:
db_dict = json.load(open(DICT_FILE_PATH,encoding='utf-8'))[0]

In [16]:
user_goals = json.load(open(USER_GOALS_FILE_PATH,encoding='utf-8'))

In [17]:
if USE_USERSIM:
    user = UserSimulator(user_goals, constants, database)
else:
    user = User(constants)
emc = ErrorModelController(db_dict, constants)
state_tracker = StateTracker(database, constants)
dqn_agent = DQNAgent(state_tracker.get_state_size(), constants)

In [18]:
warmup_run()

  0%|          | 0/1 [00:00<?, ?it/s]

Warmup Started...


41667it [19:25, 35.74it/s]           

...Warmup Ended


In [19]:
train_run()

Training Started...
Current episode: 100
episode 100: success rate: 0.73
Current episode: 200
episode 200: success rate: 0.59
Current episode: 300
episode 300: success rate: 0.69
Current episode: 400
episode 400: success rate: 0.58
Current episode: 500
episode 500: success rate: 0.57
Current episode: 600
episode 600: success rate: 0.66
Current episode: 700
episode 700: success rate: 0.64
Current episode: 800
episode 800: success rate: 0.64
Current episode: 900
episode 900: success rate: 0.52
Current episode: 1000
episode 1000: success rate: 0.67
Current episode: 1100
episode 1100: success rate: 0.62
Current episode: 1200
episode 1200: success rate: 0.61
Current episode: 1300
episode 1300: success rate: 0.67
Current episode: 1400
episode 1400: success rate: 0.77
Current episode: 1500
episode 1500: success rate: 0.69
Current episode: 1600
episode 1600: success rate: 0.67
Current episode: 1700
episode 1700: success rate: 0.66
Current episode: 1800
episode 1800: success rate: 0.54
Current 

Current episode: 14600
episode 14600: success rate: 0.63
Current episode: 14700
episode 14700: success rate: 0.7
Current episode: 14800
episode 14800: success rate: 0.8
Current episode: 14900
episode 14900: success rate: 0.82
Current episode: 15000
episode 15000: success rate: 0.7
Current episode: 15100
episode 15100: success rate: 0.77
Current episode: 15200
episode 15200: success rate: 0.7
Current episode: 15300
episode 15300: success rate: 0.86
Episode: 15300 NEW BEST SUCCESS RATE: 0.86 Avg Reward: 12.58
Current episode: 15400
episode 15400: success rate: 0.72
Current episode: 15500
episode 15500: success rate: 0.7
Current episode: 15600
episode 15600: success rate: 0.77
Current episode: 15700
episode 15700: success rate: 0.74
Current episode: 15800
episode 15800: success rate: 0.7
Current episode: 15900
episode 15900: success rate: 0.72
Current episode: 16000
episode 16000: success rate: 0.81
Current episode: 16100
episode 16100: success rate: 0.76
Current episode: 16200
episode 16

In [20]:
# constants['agent']['save_weights_file_path'] = f'weight/goal400_100_10k.h5'
# dqn_agent = DQNAgent(state_tracker.get_state_size(), constants)

In [21]:
constants

{'db_file_paths': {'database': 'data/db_hcmut_600.json',
  'dict': 'data/db_entity_jan23.json',
  'user_goals': 'data/goal_hcmut_600.json'},
 'run': {'usersim': True,
  'warmup_mem': 2600,
  'num_ep_run': 25000,
  'train_freq': 100,
  'max_round_num': 15,
  'success_rate_threshold': 0.2},
 'agent': {'save_weights_file_path': 'weight/23_01_2021_25000_100_15.h5',
  'load_weights_file_path': 'weight/23_01_2021_25000_100_15.h5',
  'vanilla': True,
  'learning_rate': 0.0005,
  'batch_size': 64,
  'dqn_hidden_size': 100,
  'epsilon_init': 0.0,
  'gamma': 0.9,
  'max_mem_size': 500000},
 'emc': {'slot_error_mode': 0,
  'slot_error_prob': 0.1,
  'intent_error_prob': 0.0}}

In [22]:
# dqn_agent.save_weights()

In [23]:
# NUM_EP_TEST = run_dict['num_ep_run']

In [24]:
NUM_EP_TEST = 500

In [25]:
def test_run():
    """
    Runs the loop that tests the agent.

    Tests the agent on the goal-oriented chatbot task. Only for evaluating a trained agent. Terminates when the episode
    reaches NUM_EP_TEST.

    """

    print('Testing Started...')
    episode = 0
    while episode < NUM_EP_TEST:
        episode_reset()
        episode += 1
        ep_reward = 0
        done = False
        # Get initial state from state tracker
        state = state_tracker.get_state()
        while not done:
            # Agent takes action given state tracker's representation of dialogue
            agent_action_index, agent_action = dqn_agent.get_action(state)
            # Update state tracker with the agent's action
            state_tracker.update_state_agent(agent_action)
            print("agent: {}".format(str(agent_action)))

            # User takes action given agent action
            user_action, reward, done, success = user.step(agent_action)
            print("user: {}".format(str(user_action)))

            ep_reward += reward
            if not done:
                # Infuse error into semantic frame level of user action
                emc.infuse_error(user_action)
            # Update state tracker with user action
            state_tracker.update_state_user(user_action)
            # Grab "next state" as state
            state = state_tracker.get_state(done)
        print('Episode: {} Success: {} Reward: {}'.format(episode, success, ep_reward))
    print('...Testing Ended')

In [26]:
test_run()

Testing Started...
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'satisfy': 'UNK'}, 'round': 1, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'satisfy': ['8.32']}}
agent: {'intent': 'inform', 'inform_slots': {'subject_group': ('d7',)}, 'request_slots': {}, 'round': 2, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'subject_group': ['d7']}}
agent: {'intent': 'inform', 'inform_slots': {'type_edu': ('đại học chính quy đại trà',)}, 'request_slots': {}, 'round': 3, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'type_edu': ['đại học chính quy đại trà']}}
agent: {'intent': 'inform', 'inform_slots': {'year': ('2020',)}, 'request_slots': {}, 'round': 4, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'year': ['2020']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'tuition': 'UNK'}, 'round': 5, 'speaker

user: {'intent': 'request', 'request_slots': {'tuition': 'UNK'}, 'inform_slots': {'subject': ['ngữ văn']}}
agent: {'intent': 'inform', 'inform_slots': {'tuition': ('320000 việt nam đồng',)}, 'request_slots': {}, 'round': 7, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'point': ['24.75']}}
agent: {'intent': 'match_found', 'inform_slots': {'major_code': ['209'], 'career': ['các chương trình ứng phó biến đổi khí hậu nước biển dâng'], 'major_name': ['kỹ thuật hóa học'], 'type_edu': ['đại học chính quy đại trà'], 'point': ['24.75'], 'subject': ['ngữ văn'], 'satisfy': ['8.2'], 'year': ['2020'], 'subject_group': ['a01'], '_id': '8441ee86-8480-44f4-9f4f-f478b4db6f37', 'tuition': ['320000 việt nam đồng'], 'major': '589'}, 'request_slots': {}, 'round': 8, 'speaker': 'Agent'}
user: {'intent': 'thanks', 'request_slots': {}, 'inform_slots': {}}
agent: {'intent': 'inform', 'inform_slots': {'career': ('các chương trình ứng phó biến đổi khí hậu nước biển dâng',)

agent: {'intent': 'inform', 'inform_slots': {'subject_group': ('d7',)}, 'request_slots': {}, 'round': 8, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'subject_group': ['d7']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_code': 'UNK'}, 'round': 9, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'major_code': ['140']}}
agent: {'intent': 'match_found', 'inform_slots': {'year': ['2019'], 'type_edu': ['phân hiệu bến tre'], 'satisfy': ['7.5'], 'point': ['19.75'], 'career': ['định hướng các điểm mấu chốt trong vận hành'], 'major_code': ['140'], 'subject_group': ['d7'], 'tuition': ['320000 việt nam đồng'], 'major_name': ['kỹ thuật xây dựng công trình thủy'], '_id': '320651c6-544e-44a9-8a50-db66f907be6b', 'subject': ['vật lý'], 'major': '111'}, 'request_slots': {}, 'round': 10, 'speaker': 'Agent'}
user: {'intent': 'thanks', 'request_slots': {}, 'inform_slots': {}}
agent: {'intent': 'done',

agent: {'intent': 'done', 'inform_slots': {}, 'request_slots': {}, 'round': 12, 'speaker': 'Agent'}
user: {'intent': 'done', 'request_slots': {}, 'inform_slots': {}}
Episode: 20 Success: False Reward: -27
agent: {'intent': 'inform', 'inform_slots': {'subject': ('vật lý',)}, 'request_slots': {}, 'round': 1, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'subject': ['ngữ văn']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'subject_group': 'UNK'}, 'round': 2, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'subject_group': ['d1']}}
agent: {'intent': 'inform', 'inform_slots': {'type_edu': ('phân hiệu bến tre',)}, 'request_slots': {}, 'round': 3, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'type_edu': ['phân hiệu bến tre']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_code': 'UNK'}, 'round': 4, 'speaker': 'Agent'}
user: {'in

agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_code': 'UNK'}, 'round': 7, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'major_code': ['245']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'tuition': 'UNK'}, 'round': 8, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'tuition': ['1850000 việt nam đồng']}}
agent: {'intent': 'inform', 'inform_slots': {'major_name': ('kỹ thuật trắc địa bản đồ',)}, 'request_slots': {}, 'round': 9, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'point': ['25.0']}}
agent: {'intent': 'inform', 'inform_slots': {'satisfy': ('8.65',)}, 'request_slots': {}, 'round': 10, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'satisfy': ['8.65']}}
agent: {'intent': 'match_found', 'inform_slots': {'career': ['khảo sát'], 'satisfy': ['8.65'], 'major_code': ['245'], 'year': ['20

...

agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'tuition': 'UNK'}, 'round': 8, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'tuition': ['1850000 việt nam đồng']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'point': 'UNK'}, 'round': 9, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'point': ['19.75']}}
agent: {'intent': 'inform', 'inform_slots': {'major_name': 'no match available'}, 'request_slots': {}, 'round': 10, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'career': ['duy trì sản xuất liên tục và cung cấp dịch vụ bảo trì hậu mãi']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'career': 'UNK'}, 'round': 11, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'career': ['duy trì sản xuất liên tục và cung cấp dịch vụ bảo trì hậu mãi']}}
agent: {'intent': 'match_found', 'in

agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'type_edu': 'UNK'}, 'round': 1, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'type_edu': ['phân hiệu bến tre']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'point': 'UNK'}, 'round': 2, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'point': ['26.75']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_name': 'UNK'}, 'round': 3, 'speaker': 'Agent'}
user: {'intent': 'request', 'request_slots': {'major_name': 'UNK'}, 'inform_slots': {'subject_group': ['v']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'year': 'UNK'}, 'round': 4, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'year': ['2020']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_name': 'UNK'}, 'round': 5, 'speaker': 'Agent'}
user: {'intent': 'request',

agent: {'intent': 'inform', 'inform_slots': {'point': ('19.0',)}, 'request_slots': {}, 'round': 12, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'point': ['19.0']}}
agent: {'intent': 'done', 'inform_slots': {}, 'request_slots': {}, 'round': 13, 'speaker': 'Agent'}
user: {'intent': 'done', 'request_slots': {}, 'inform_slots': {}}
Episode: 481 Success: True Reward: 9.5
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'satisfy': 'UNK'}, 'round': 1, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'satisfy': ['8.65']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'tuition': 'UNK'}, 'round': 2, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'tuition': ['320000 việt nam đồng']}}
agent: {'intent': 'inform', 'inform_slots': {'subject': ('hóa học',)}, 'request_slots': {}, 'round': 3, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slot

agent: {'intent': 'done', 'inform_slots': {}, 'request_slots': {}, 'round': 14, 'speaker': 'Agent'}
user: {'intent': 'done', 'request_slots': {}, 'inform_slots': {}}
Episode: 489 Success: False Reward: -29
agent: {'intent': 'inform', 'inform_slots': {'type_edu': ('phân hiệu bến tre',)}, 'request_slots': {}, 'round': 1, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'type_edu': ['phân hiệu bến tre']}}
agent: {'intent': 'inform', 'inform_slots': {'subject_group': ('v',)}, 'request_slots': {}, 'round': 2, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'subject_group': ['a']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_name': 'UNK'}, 'round': 3, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'major_name': ['công nghệ sinh học']}}
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'type_edu': 'UNK'}, 'round': 4, 'speaker': 'Agent'}


agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'major_name': 'UNK'}, 'round': 15, 'speaker': 'Agent'}
user: {'intent': 'done', 'request_slots': {}, 'inform_slots': {}}
Episode: 496 Success: False Reward: -30
agent: {'intent': 'request', 'inform_slots': {}, 'request_slots': {'subject': 'UNK'}, 'round': 1, 'speaker': 'Agent'}
user: {'intent': 'request', 'request_slots': {'subject': 'UNK'}, 'inform_slots': {'career': ['thiết kế sản phẩm cơ khí']}}
agent: {'intent': 'inform', 'inform_slots': {'subject': ('vẽ hình họa mỹ thuật',)}, 'request_slots': {}, 'round': 2, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'major_code': ['114']}}
agent: {'intent': 'inform', 'inform_slots': {'type_edu': ('chương trình chất lượng cao tiên tiến',)}, 'request_slots': {}, 'round': 3, 'speaker': 'Agent'}
user: {'intent': 'inform', 'request_slots': {}, 'inform_slots': {'type_edu': ['chương trình chất lượng cao tiên tiến']}}
agent: {'intent': 'request', 